In [1]:
import dask.dataframe as dd

In [2]:

# Carregar a tabela com d_x colunas
sales = dd.read_csv('../data/item_sales.csv')

In [3]:
# Identificar todas as colunas que começam com 'd_'
d_columns = [col for col in sales.columns if col.startswith('d_')]

# Exibir as primeiras 10 colunas encontradas para verificação
print(f"Colunas de dias encontradas: {d_columns[:10]}...")

Colunas de dias encontradas: ['d_1', 'd_2', 'd_3', 'd_4', 'd_5', 'd_6', 'd_7', 'd_8', 'd_9', 'd_10']...


In [4]:
def melt_sales(partition):
    return partition.melt(
        id_vars=['id', 'item', 'category', 'department', 'store', 'store_code', 'region'],
        value_vars=d_columns,
        var_name='d',
        value_name='sales'
    )

In [5]:
# Aplicar a função melt_sales em cada partição
sales_long = sales.map_partitions(
    melt_sales,
    meta={
        'id': 'object',
        'item': 'object',
        'category': 'object',
        'department': 'object',
        'store': 'object',
        'store_code': 'object',
        'region': 'object',
        'd': 'object',
        'sales': 'int32'
    }
)

In [6]:
# Reparticionar o DataFrame para ter mais partições menores
sales_long = sales_long.repartition(npartitions=100)

In [7]:
# Salvar como múltiplos arquivos Parquet
sales_long.to_parquet('../data/sales/', write_index=False)


In [8]:
sales_long.head()

,id,item,category,department,store,store_code,region,d,sales
0,ACCESORIES_1_001_NYC_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1,0
1,ACCESORIES_1_002_NYC_1,ACCESORIES_1_002,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1,0
2,ACCESORIES_1_003_NYC_1,ACCESORIES_1_003,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1,0
3,ACCESORIES_1_004_NYC_1,ACCESORIES_1_004,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1,0
4,ACCESORIES_1_005_NYC_1,ACCESORIES_1_005,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1,0
